In [1]:
import os
import copy
import numpy as np
import random
import torch
import torch.nn.functional as F
from torch.utils.data import Subset
import torch.nn as nn
from torchvision import datasets, transforms
from PIL import Image
from model_resnet_official import ResNet50
import torchvision.models as models
import time
import torchvision
from torch.cuda.amp import autocast as autocast

In [2]:
# torch settings
seed = 1
torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.cuda.manual_seed_all(1)
np.random.seed(1)
random.seed(1)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

# TODO:Hyperparameters Setting
# Main function
rootpath = "./record/"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
datasetType = 'clothing1m'
num_classes = 14
model_name = 'resnet50'
num_users = 100
total_rounds = 40

# TODO:Load Data
# TODO: Load da

In [3]:
class Clothing(torch.utils.data.Dataset):
    def __init__(self, root, transform, mode):
        self.root = root
        self.noisy_labels = {}
        self.clean_labels = {}
        self.data = []
        self.targets = []
        self.transform = transform
        self.mode = mode

        with open(self.root + 'noisy_label_kv.txt', 'r') as f:
            lines = f.read().splitlines()
        for l in lines:
            entry = l.split()
            img_path = self.root + entry[0]
            self.noisy_labels[img_path] = int(entry[1])

        with open(self.root + 'clean_label_kv.txt', 'r') as f:
            lines = f.read().splitlines()
        for l in lines:
            entry = l.split()
            img_path = self.root + entry[0]
            self.clean_labels[img_path] = int(entry[1])

        if self.mode == 'train':
            with open(self.root + 'noisy_train_key_list.txt', 'r') as f:
                lines = f.read().splitlines()
            for l in lines:
                img_path = self.root + l
                self.data.append(img_path)
                target = self.noisy_labels[img_path]
                self.targets.append(target)
        elif self.mode == 'minitrain':
            with open(self.root + 'noisy_train_key_list.txt', 'r') as f:
                lines = f.read().splitlines()
            n = len(lines)
            np.random.seed(13)
            subset_idx = np.random.choice(n, int(n/10), replace=False)
            for i in subset_idx:
                l = lines[i]
                img_path = self.root + l
                self.data.append(img_path)
                target = self.noisy_labels[img_path]
                self.targets.append(target)
        elif self.mode == 'test':
            with open(self.root + 'clean_test_key_list.txt', 'r') as f:
                lines = f.read().splitlines()
            for l in lines:
                img_path = self.root + l
                self.data.append(img_path)
                target = self.clean_labels[img_path]
                self.targets.append(target)

    def __getitem__(self, index):
        img_path = self.data[index]
        target = self.targets[index]
        image = Image.open(img_path).convert('RGB')
        img = self.transform(image)
        return img, target

    def __len__(self):
        return len(self.data)


def iid_sampling(n_train, num_users, seed):
    np.random.seed(seed)
    num_items = int(n_train/num_users)
    # initial user and index for whole dataset
    ddict_users, all_idxs = {}, [i for i in range(n_train)]
    for i in range(num_users):
        # 'replace=False' make sure that there is no repeat
        ddict_users[i] = set(np.random.choice(
            all_idxs, num_items, replace=False))
        all_idxs = list(set(all_idxs)-ddict_users[i])
    return ddict_users


def get_dataset(datasetType):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #TODO: remember to put your dataset in this directory
    if datasetType == 'clothing1m':
        data_path = './data/clothing1M/'
        trans_train = transforms.Compose([
            transforms.Resize((224, 224)),
#             transforms.RandomCrop(224),
#             transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[
                                 0.229, 0.224, 0.225]),
        ])
        trans_val = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[
                                 0.229, 0.224, 0.225]),
        ])
        dataset_train = Clothing(data_path, trans_train, "train")
        dataset_test = Clothing(data_path, trans_val, "test")
        n_train = len(dataset_train)
        y_train = np.array(dataset_train.targets)
    else:
        exit('Error: unrecognized dataset')

    dict_users = iid_sampling(n_train, num_users, seed)

    return dataset_train, dataset_test, dict_users


In [4]:
def build_model():
    # choose different Neural network model for different args or input
    netglob = ResNet50(pretrained=False)
    # if args.pretrained:
    model = models.resnet50(pretrained=True)
    netglob.load_state_dict(model.state_dict())
    netglob.fc = nn.Linear(2048, num_classes)
    netglob = netglob.to(device)

    return netglob

#w_glob_fl = FedAvg(w_locals, dict_len)


def Averaging(w, dict_len):
    w_avg = copy.deepcopy(w[0])
    for k in w_avg.keys():
        w_avg[k] = w_avg[k] * dict_len[0]
        for i in range(1, len(w)):
            w_avg[k] += w[i][k] * dict_len[i]
        w_avg[k] = w_avg[k] / sum(dict_len)
    return w_avg


s=1
color_jitter = torchvision.transforms.ColorJitter(0.8 * s, 0.8 * s, 0.8 * s, 0.2 * s)

tt_transform = transforms.Compose([
#         transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomApply([color_jitter], p=0.8),
#         transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
        transforms.RandomGrayscale(p=0.2)])


class LocalUpdate(object):
    def __init__(self, dataset, learning_rate, data_idxs):
        # self.client_index = client_index
        self.data_idxs = data_idxs
        self.batchsize = 64
        dataset_client = Subset(dataset, data_idxs) #modify
        self.train_loader = torch.utils.data.DataLoader(
            dataset=dataset_client, batch_size=self.batchsize, shuffle=True,drop_last=True,num_workers=4,pin_memory=True)
        self.learning_rate = learning_rate
        self.momentum = 0.9
        self.epochs = 5
        self.weight_decay = 0.0005
        self.local_datasize = len(data_idxs)
    
    def L1loss(self,predict,target):
        loss_f = nn.L1Loss(reduction='mean')
        l1_loss = loss_f(predict,target)
        return l1_loss
    
    def L2loss(self,predict,target):
        loss_f_mse = nn.MSELoss(reduction='mean')
        loss_mse = loss_f_mse(predict,target)
        return loss_mse

    def js(self, p_output, q_output):
        """
        Function that measures JS divergence between target and output logits:
        """
        KLDivLoss = nn.KLDivLoss(reduction='batchmean')
        log_mean_output = ((p_output + q_output )/2).log()
        return (KLDivLoss(log_mean_output, p_output) + KLDivLoss(log_mean_output, q_output))/2
    
    def train(self, model, current):
        criterion = nn.CrossEntropyLoss()
        e_loss = []
        model.cuda()
        for _ in range(1, self.epochs+1):  # _ means epoch
            train_loss = 0.0
            model.train()
            for data, labels in self.train_loader:
                if torch.cuda.is_available():
                    data, labels = data.cuda(non_blocking=True), labels.cuda(non_blocking=True)
#                     data_aug = tt_transform(data).cuda(non_blocking=True)   
                    
                optimizer = torch.optim.SGD(model.parameters(
                    ), lr=self.learning_rate, momentum=self.momentum, weight_decay=self.weight_decay)
                optimizer.zero_grad()  # clear the gradients
                
                mix_1 = np.random.beta(1,1)
                mix_2 = 1-mix_1
                
                # herein we exploit the mixed precision training technique to accelerate the FL process
                with autocast():
                    output = model(data)  # make a forward pass

                    loss = criterion(output, labels)   # calculate the loss
                
                loss.backward()  # do a backwards pass
                optimizer.step()  # perform a single optimization step
                train_loss += loss.item()*data.size(0)  # update training loss

            # average losses
            train_loss = train_loss/len(self.train_loader.dataset)
            print(f"Train for this client epoch {_} , loss {train_loss}")
            e_loss.append(train_loss)

        total_loss = sum(e_loss)/len(e_loss)
        return model.state_dict(), total_loss


In [5]:
print(device)
dataset_train, dataset_test, dict_users = get_dataset('clothing1m')
# build model
netglob = build_model()
net_local = build_model()
netglob = netglob.cuda()
net_local = net_local.cuda()

cuda:0


In [6]:
test_loader = torch.utils.data.DataLoader(
        dataset=dataset_test, batch_size=256, shuffle=False,num_workers=4,pin_memory=True)

def globaltest(net):
    print("Start Testing")
    net.cuda()
    net.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images, labels = images.cuda(non_blocking=True), labels.cuda(non_blocking=True)
            outputs = net(images)
            # outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = correct / total
    print(f"acc {acc}")
    return acc

In [7]:
# client participation rate
C = 0.05

Clients = []
for idx in range(num_users):
    sample_idx = np.array(list(dict_users[idx]))
    cl = LocalUpdate(dataset=dataset_train,learning_rate=0.01, data_idxs=sample_idx)
    Clients.append(cl)
    print(f"Generate client {idx}")
    
    
    
print("Initialize Done")
f_acc = open(datasetType + '_fedavg_acc.txt', 'a')
acc_list = []
for _ in range(total_rounds):
    idxs_users = np.random.choice(num_users, int(num_users*C),replace=False)
    print(f"Round {_} selects {idxs_users}")
    
    # selected clients' model weights of each rounds
    w_locals = []
    
    
    # local training for each selected client
    for idx in idxs_users:
        net_local.load_state_dict(netglob.state_dict())
        local = Clients[idx]
        w, loss = local.train(model=copy.deepcopy(net_local).to(device),current=_)
        w_locals.append(copy.deepcopy(w))

    dict_len = [len(dict_users[idx]) for idx in idxs_users]
    print(f"start aggregation for round {_}")
    w_glob = Averaging(w_locals, dict_len)

    netglob.load_state_dict(copy.deepcopy(w_glob))
    acc_t = globaltest(copy.deepcopy(netglob).to(device))
    acc_list.append(acc_t)
    print(acc_list)
    f_acc.write("global round %d ,acc: %.4f \n" % (_, acc_t))
    # namep = str(_)+'_fedavg_round.pt'
    # torch.save(netglob.state_dict(),namep)


Generate client 0
Generate client 1
Generate client 2
Generate client 3
Generate client 4
Generate client 5
Generate client 6
Generate client 7
Generate client 8
Generate client 9
Generate client 10
Generate client 11
Generate client 12
Generate client 13
Generate client 14
Generate client 15
Generate client 16
Generate client 17
Generate client 18
Generate client 19
Generate client 20
Generate client 21
Generate client 22
Generate client 23
Generate client 24
Generate client 25
Generate client 26
Generate client 27
Generate client 28
Generate client 29
Generate client 30
Generate client 31
Generate client 32
Generate client 33
Generate client 34
Generate client 35
Generate client 36
Generate client 37
Generate client 38
Generate client 39
Generate client 40
Generate client 41
Generate client 42
Generate client 43
Generate client 44
Generate client 45
Generate client 46
Generate client 47
Generate client 48
Generate client 49
Generate client 50
Generate client 51
Generate client 52
Gen